In [4]:
import numpy as np

# --- Load saved parameters ---
# Update the filepath as necessary (e.g., "model_params_30000.npz")

params = np.load("D:\\ASCC_parts_extended\\workspace\\model_params_20000.npy.npz")

# Extract gaussian parameters (they were saved under these keys)
means = params["gauss_params.means"]   # shape: (N, 3)
scales = params["gauss_params.scales"]   # shape: (N, 3) -- these are log scales!
quats = params["gauss_params.quats"]     # shape: (N, 4)

# For demonstration purposes, we use only the first 100 gaussians.
# num_gaussians = 100
# means = means[:num_gaussians]
# scales = scales[:num_gaussians]
# quats = quats[:num_gaussians]

# Convert log-scales to radii (using elementwise exp)
radii = np.exp(scales)  # Each gaussian now has radii along x, y, z

# --- Helper functions ---

def quat_to_rot_matrix(q):
    """
    Convert a quaternion to a 3x3 rotation matrix.
    Assumes quaternion format is (w, x, y, z).
    """
    w, x, y, z = q
    R = np.array([
        [1 - 2*y**2 - 2*z**2,   2*x*y - 2*z*w,       2*x*z + 2*y*w],
        [2*x*y + 2*z*w,         1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w,         2*y*z + 2*x*w,       1 - 2*x**2 - 2*y**2]
    ])
    return R

# Compute rotation matrices for each gaussian from its quaternion.
rot_matrices = np.array([quat_to_rot_matrix(q) for q in quats])

def ray_ellipsoid_intersection(ray_origin, ray_dir, center, radii, rot_matrix):
    """
    Compute the intersection of a ray with an ellipsoid.
    The ellipsoid is defined by:
        (R^T (x - center))^2 / radii^2 = 1
    where R is the rotation matrix of the ellipsoid.
    
    Parameters:
        ray_origin : (3,) array, origin of the ray.
        ray_dir    : (3,) array, unit direction of the ray.
        center     : (3,) array, center of the ellipsoid.
        radii      : (3,) array, radii (semi-axis lengths) of the ellipsoid.
        rot_matrix : (3,3) array, rotation matrix of the ellipsoid.
        
    Returns:
        The smallest positive intersection distance t (such that ray_origin + t*ray_dir is on the ellipsoid),
        or None if there is no intersection.
    """
    # Transform the ray into the ellipsoid's local coordinate system.
    p = ray_origin - center
    p_local = rot_matrix.T @ p
    d_local = rot_matrix.T @ ray_dir

    # Solve quadratic: A*t^2 + B*t + C = 0 where
    A = np.sum((d_local / radii)**2)
    B = 2 * np.sum((p_local * d_local) / (radii**2))
    C = np.sum((p_local / radii)**2) - 1

    discriminant = B**2 - 4 * A * C
    if discriminant < 0:
        return None  # No real intersection.
    
    sqrt_disc = np.sqrt(discriminant)
    t1 = (-B - sqrt_disc) / (2 * A)
    t2 = (-B + sqrt_disc) / (2 * A)
    # We choose the smallest positive t.
    t_candidates = [t for t in (t1, t2) if t > 0]
    if not t_candidates:
        return None
    return min(t_candidates)

# --- Set up test rays ---
# For simplicity, we use rays originating at (0, 0, 0) and pointing in three different directions.
ray_origins = np.array([[0, 0, 0]] * 3)
ray_dirs = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])
# Normalize the ray directions (in case they aren't unit vectors already)
ray_dirs = ray_dirs / np.linalg.norm(ray_dirs, axis=1, keepdims=True)

# --- Compute intersections ---
for i, (origin, direction) in enumerate(zip(ray_origins, ray_dirs)):
    closest_t = np.inf
    closest_gaussian = None
    for j in range(num_gaussians):
        t = ray_ellipsoid_intersection(origin, direction, means[j], radii[j], rot_matrices[j])
        if t is not None and t < closest_t:
            closest_t = t
            closest_gaussian = j
    if closest_gaussian is not None:
        print(f"Ray {i} (direction: {direction}) intersects gaussian {closest_gaussian} at distance {closest_t:.3f}")
    else:
        print(f"Ray {i} (direction: {direction}) did not intersect any gaussian.")


Ray 0 (direction: [1. 0. 0.]) did not intersect any gaussian.
Ray 1 (direction: [0. 1. 0.]) did not intersect any gaussian.
Ray 2 (direction: [0. 0. 1.]) did not intersect any gaussian.


In [6]:
!python -m pip install -U scikit-image

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.8 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 814.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.2/12.9 MB 7.6 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/12.9 MB 13.3 MB/s eta 0:00:01
   -------- ------------------------------- 2.7/12.9 MB 21.9 MB/s eta 0:00:01
   -------------- ------------------------- 4.6/12.9 MB 26.8 MB/s eta 0:00:01
   -------------------- ------------------- 6.5/12.9 MB 29.6 MB/s eta 0:00:01
   ------------------------- -------------- 8.4/12.9 MB 31.4 MB/s eta 0:00:01
   ------------------------------- -------- 10.2/12.9 MB 32.7 MB/s eta 0:00:01
   ------------------------------------- -- 12.1/12.9 MB 40.9 MB/s eta 0:00:01


In [8]:
import numpy as np
from skimage import measure
import open3d as o3d
import numpy as np

# --- Step 1: Load the Gaussian Parameters ---

# Update the filepath as needed.
params = np.load("D:\\ASCC_parts_extended\\workspace\\model_params_20000.npy.npz")
means = params["gauss_params.means"]   # shape: (N, 3)
scales_log = params["gauss_params.scales"]   # shape: (N, 3) (log-scales)
quats = params["gauss_params.quats"]     # shape: (N, 4)

# For this demo, use all gaussians.
N = means.shape[0]

# Convert log-scales to actual scales (radii per axis).
scales = np.exp(scales_log)  # shape: (N, 3)

# --- Helper: Convert quaternion to rotation matrix ---
def quat_to_rot_matrix(q):
    """
    Convert a quaternion (assumed order: w, x, y, z) to a 3x3 rotation matrix.
    """
    w, x, y, z = q
    R = np.array([
        [1 - 2*y**2 - 2*z**2,   2*x*y - 2*z*w,       2*x*z + 2*y*w],
        [2*x*y + 2*z*w,         1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w,         2*y*z + 2*x*w,       1 - 2*x**2 - 2*y**2]
    ])
    return R

# Compute rotation matrices for all gaussians.
rot_matrices = np.array([quat_to_rot_matrix(q) for q in quats])  # shape: (N, 3, 3)

# --- Step 2: Build the Density Field ---

# Compute a bounding box that covers all gaussians.
# We use each mean extended by its scale (radii) as a rough estimate.
min_bound = np.min(means - scales, axis=0)
max_bound = np.max(means + scales, axis=0)
# Add a little margin.
margin = 0.1 * (max_bound - min_bound)
min_bound -= margin
max_bound += margin

# Define grid resolution. Adjust (nx, ny, nz) for finer or coarser resolution.
nx, ny, nz = 128, 128, 128
x = np.linspace(min_bound[0], max_bound[0], nx)
y = np.linspace(min_bound[1], max_bound[1], ny)
z = np.linspace(min_bound[2], max_bound[2], nz)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
grid_points = np.stack([X, Y, Z], axis=-1)  # shape: (nx, ny, nz, 3)

# Initialize the density grid.
density = np.zeros((nx, ny, nz), dtype=np.float32)

print("Building density field from {} gaussians on a {} grid...".format(N, density.shape))
# For each gaussian, add its contribution.
for i in range(N):
    mu = means[i]        # center, shape: (3,)
    R = rot_matrices[i]  # rotation matrix, shape: (3,3)
    sigma = scales[i]    # radii along each axis, shape: (3,)
    
    # Compute the difference from the gaussian center for all grid points.
    diff = grid_points - mu  # shape: (nx, ny, nz, 3)
    
    # Transform the differences into the Gaussian's local coordinate system.
    # Use tensordot: (nx, ny, nz, 3) dot (3,3) gives (nx, ny, nz, 3)
    diff_local = np.tensordot(diff, R.T, axes=([3], [0]))
    
    # Compute the normalized squared distance: sum_j (diff_local_j^2 / sigma_j^2)
    norm2 = np.sum((diff_local ** 2) / (sigma**2), axis=-1)
    
    # Gaussian contribution (without normalization constant)
    contrib = np.exp(-0.5 * norm2)
    
    density += contrib
    if i % (N // 10 + 1) == 0:
        print(f"Processed {i+1}/{N} gaussians...")

print("Density field built.")

# --- Step 3: Extract Surface via Marching Cubes ---

# Set an isovalue threshold. You may need to adjust this value.
iso_threshold = 0.5
# Compute voxel spacing.
spacing = ( (max_bound - min_bound) / (np.array([nx, ny, nz]) - 1) )
print("Running marching cubes...")
verts, faces, normals, values = measure.marching_cubes(density, level=iso_threshold, spacing=spacing)
# Convert vertices from grid coordinates to world coordinates by adding min_bound.
verts_world = verts + min_bound

# --- Step 4: Visualize the Mesh using Open3D ---
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts_world)
mesh.triangles = o3d.utility.Vector3iVector(faces)
mesh.compute_vertex_normals()

print("Displaying mesh...")
o3d.visualization.draw_geometries([mesh])


Building density field from 1000000 gaussians on a (128, 128, 128) grid...
Processed 1/1000000 gaussians...


KeyboardInterrupt: 

In [10]:
!pip install numba

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
    --------------------------------------- 0.0/2.8 MB 487.6 kB/s eta 0:00:06
   --- ------------------------------------ 0.2/2.8 MB 1.9 MB/s eta 0:00:02
   -------------- ------------------------- 1.0/2.8 MB 6.5 MB/s eta 0:00:01
   -------------------------------------- - 2.7/2.8 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 12.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   -- ------------------------------------- 1.6/30.3 MB 50.5 MB/s eta 0:00:01
   ---- ----------------------------------- 3.4/30.3 MB 43.3 MB/s eta 0:00:01
   ------ --------------------------------- 5.3/30.3 MB 42.2 MB/s eta 0:00:01
   --------- ------------------------------ 7.2/30.3 MB 42.0 MB/s eta 0:00:01
   ------------ --------------------------- 9.1/30.3 MB 41.5 MB/s eta 0:00:01
   -------------- 

In [ ]:
import numpy as np
from numba import njit, prange
from skimage import measure
import open3d as o3d

# --- Step 1: Load the Gaussian Parameters ---
# Update the filepath as needed.
params = np.load("D:\\ASCC_parts_extended\\workspace\\model_params_20000.npy.npz")
means = params["gauss_params.means"]   # shape: (N, 3)
scales_log = params["gauss_params.scales"]   # shape: (N, 3) (log-scales)
quats = params["gauss_params.quats"]     # shape: (N, 4)

N = means.shape[0]
# Convert log-scales to actual scales.
scales = np.exp(scales_log)  # shape: (N, 3)

# --- Helper: Convert quaternion to rotation matrix ---
def quat_to_rot_matrix(q):
    """
    Convert a quaternion (assumed order: w, x, y, z) to a 3x3 rotation matrix.
    """
    w, x, y, z = q
    R = np.array([
        [1 - 2*y**2 - 2*z**2,   2*x*y - 2*z*w,       2*x*z + 2*y*w],
        [2*x*y + 2*z*w,         1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w,         2*y*z + 2*x*w,       1 - 2*x**2 - 2*y**2]
    ], dtype=np.float32)
    return R

# Compute rotation matrices for all gaussians.
rot_matrices = np.array([quat_to_rot_matrix(q) for q in quats], dtype=np.float32)

# --- Step 2: Set Up the 3D Grid ---
# Compute a bounding box that covers all gaussians.
min_bound = np.min(means - scales, axis=0)
max_bound = np.max(means + scales, axis=0)
# Add a margin.
margin = 0.1 * (max_bound - min_bound)
min_bound -= margin
max_bound += margin

# Define grid resolution. Adjust (nx, ny, nz) as needed.
nx, ny, nz = 128, 128, 128
x = np.linspace(min_bound[0], max_bound[0], nx, dtype=np.float32)
y = np.linspace(min_bound[1], max_bound[1], ny, dtype=np.float32)
z = np.linspace(min_bound[2], max_bound[2], nz, dtype=np.float32)

# Initialize the density grid.
density = np.zeros((nx, ny, nz), dtype=np.float32)

# --- Step 3: Accelerated Density Field Construction with Numba ---
@njit(parallel=True)
def add_gaussians_to_density(density, means, scales, rot_matrices, x, y, z, threshold_factor=3.0):
    nx, ny, nz = density.shape
    N = means.shape[0]
    for i in prange(N):
        # Gaussian parameters for the i-th Gaussian.
        mu0 = means[i, 0]
        mu1 = means[i, 1]
        mu2 = means[i, 2]
        sigma0 = scales[i, 0]
        sigma1 = scales[i, 1]
        sigma2 = scales[i, 2]
        # Define an influence radius: threshold_factor * max(sigma)
        r = threshold_factor * max(sigma0, sigma1, sigma2)
        
        # Determine grid indices corresponding to the bounding box [mu - r, mu + r]
        ix_min = 0
        while ix_min < nx and x[ix_min] < mu0 - r:
            ix_min += 1
        ix_max = nx - 1
        while ix_max >= 0 and x[ix_max] > mu0 + r:
            ix_max -= 1
        iy_min = 0
        while iy_min < ny and y[iy_min] < mu1 - r:
            iy_min += 1
        iy_max = ny - 1
        while iy_max >= 0 and y[iy_max] > mu1 + r:
            iy_max -= 1
        iz_min = 0
        while iz_min < nz and z[iz_min] < mu2 - r:
            iz_min += 1
        iz_max = nz - 1
        while iz_max >= 0 and z[iz_max] > mu2 + r:
            iz_max -= 1
        
        # Retrieve rotation matrix components for the i-th Gaussian.
        R00 = rot_matrices[i, 0, 0]
        R01 = rot_matrices[i, 0, 1]
        R02 = rot_matrices[i, 0, 2]
        R10 = rot_matrices[i, 1, 0]
        R11 = rot_matrices[i, 1, 1]
        R12 = rot_matrices[i, 1, 2]
        R20 = rot_matrices[i, 2, 0]
        R21 = rot_matrices[i, 2, 1]
        R22 = rot_matrices[i, 2, 2]
        
        # Loop only over the subregion of the grid influenced by the Gaussian.
        for ix in range(ix_min, ix_max + 1):
            for iy in range(iy_min, iy_max + 1):
                for iz in range(iz_min, iz_max + 1):
                    # World coordinate of the current voxel.
                    pos0 = x[ix]
                    pos1 = y[iy]
                    pos2 = z[iz]
                    diff0 = pos0 - mu0
                    diff1 = pos1 - mu1
                    diff2 = pos2 - mu2
                    # Transform the difference into the Gaussian’s local coordinate system:
                    # local = R^T * diff.
                    local0 = R00 * diff0 + R10 * diff1 + R20 * diff2
                    local1 = R01 * diff0 + R11 * diff1 + R21 * diff2
                    local2 = R02 * diff0 + R12 * diff1 + R22 * diff2
                    # Compute the normalized squared distance.
                    norm2 = (local0**2)/(sigma0**2) + (local1**2)/(sigma1**2) + (local2**2)/(sigma2**2)
                    density[ix, iy, iz] += np.exp(-0.5 * norm2)

print("Building density field with Numba...")
add_gaussians_to_density(density, means.astype(np.float32), scales.astype(np.float32), rot_matrices, x, y, z)
print("Density field built.")

# --- Step 4: Surface Extraction via Marching Cubes ---
spacing = ( (max_bound - min_bound) / (np.array([nx, ny, nz]) - 1) )
print("Running marching cubes...")
verts, faces, normals, values = measure.marching_cubes(density, level=0.5, spacing=spacing)
# Convert vertices to world coordinates.
verts_world = verts + min_bound

# --- Step 5: Visualize the Mesh using Open3D ---
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts_world)
mesh.triangles = o3d.utility.Vector3iVector(faces)
mesh.compute_vertex_normals()

print("Displaying mesh...")
o3d.visualization.draw_geometries([mesh])


Building density field with Numba...
Density field built.
Running marching cubes...


In [4]:
import numpy as np
from numba import njit, prange
from skimage import measure
import open3d as o3d
import ipywidgets as widgets
from IPython.display import display

# --- Step 1: Load the Gaussian Parameters ---
# Update the filepath as needed.
params = np.load("D:\\ASCC_parts_extended\\workspace\\model_params_20000.npy.npz")
means = params["gauss_params.means"]   # shape: (N, 3)
scales_log = params["gauss_params.scales"]   # shape: (N, 3)
quats = params["gauss_params.quats"]     # shape: (N, 4)

N = means.shape[0]
# Convert log-scales to actual scales.
scales = np.exp(scales_log)  # shape: (N, 3)

# --- Helper: Convert quaternion to rotation matrix ---
def quat_to_rot_matrix(q):
    """
    Convert a quaternion (assumed order: w, x, y, z) to a 3x3 rotation matrix.
    """
    w, x, y, z = q
    R = np.array([
        [1 - 2*y**2 - 2*z**2,   2*x*y - 2*z*w,       2*x*z + 2*y*w],
        [2*x*y + 2*z*w,         1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w,         2*y*z + 2*x*w,       1 - 2*x**2 - 2*y**2]
    ], dtype=np.float32)
    return R

# Compute rotation matrices for all gaussians.
rot_matrices = np.array([quat_to_rot_matrix(q) for q in quats], dtype=np.float32)

# --- Step 2: Set Up the 3D Grid ---
# Compute a bounding box that covers all gaussians.
min_bound = np.min(means - scales, axis=0)
max_bound = np.max(means + scales, axis=0)
# Add a margin.
margin = 0.1 * (max_bound - min_bound)
min_bound -= margin
max_bound += margin

# Define grid resolution.
nx, ny, nz = 128, 128, 128
x = np.linspace(min_bound[0], max_bound[0], nx, dtype=np.float32)
y = np.linspace(min_bound[1], max_bound[1], ny, dtype=np.float32)
z = np.linspace(min_bound[2], max_bound[2], nz, dtype=np.float32)
# Pre-calculate spacing for marching cubes.
spacing = ( (max_bound - min_bound) / (np.array([nx, ny, nz]) - 1) )

# --- Step 3: Accelerated Density Field Construction with Numba ---
@njit
def add_gaussians_to_density(density, means, scales, rot_matrices, x, y, z, threshold_factor=3.0):
    nx, ny, nz = density.shape
    N = means.shape[0]
    for i in range(N):
        # Gaussian parameters for the i-th Gaussian.
        mu0 = means[i, 0]
        mu1 = means[i, 1]
        mu2 = means[i, 2]
        sigma0 = scales[i, 0]
        sigma1 = scales[i, 1]
        sigma2 = scales[i, 2]
        # Define an influence radius: threshold_factor * max(sigma)
        r = threshold_factor * max(sigma0, sigma1, sigma2)
        
        # Determine grid indices corresponding to the bounding box [mu - r, mu + r]
        ix_min = 0
        while ix_min < nx and x[ix_min] < mu0 - r:
            ix_min += 1
        ix_max = nx - 1
        while ix_max >= 0 and x[ix_max] > mu0 + r:
            ix_max -= 1
        iy_min = 0
        while iy_min < ny and y[iy_min] < mu1 - r:
            iy_min += 1
        iy_max = ny - 1
        while iy_max >= 0 and y[iy_max] > mu1 + r:
            iy_max -= 1
        iz_min = 0
        while iz_min < nz and z[iz_min] < mu2 - r:
            iz_min += 1
        iz_max = nz - 1
        while iz_max >= 0 and z[iz_max] > mu2 + r:
            iz_max -= 1
        
        # Retrieve rotation matrix components for the i-th Gaussian.
        R00 = rot_matrices[i, 0, 0]
        R01 = rot_matrices[i, 0, 1]
        R02 = rot_matrices[i, 0, 2]
        R10 = rot_matrices[i, 1, 0]
        R11 = rot_matrices[i, 1, 1]
        R12 = rot_matrices[i, 1, 2]
        R20 = rot_matrices[i, 2, 0]
        R21 = rot_matrices[i, 2, 1]
        R22 = rot_matrices[i, 2, 2]
        
        # Loop only over the subregion of the grid influenced by the Gaussian.
        for ix in range(ix_min, ix_max + 1):
            for iy in range(iy_min, iy_max + 1):
                for iz in range(iz_min, iz_max + 1):
                    # World coordinate of the current voxel.
                    pos0 = x[ix]
                    pos1 = y[iy]
                    pos2 = z[iz]
                    diff0 = pos0 - mu0
                    diff1 = pos1 - mu1
                    diff2 = pos2 - mu2
                    # Transform the difference into the Gaussian’s local coordinate system:
                    local0 = R00 * diff0 + R10 * diff1 + R20 * diff2
                    local1 = R01 * diff0 + R11 * diff1 + R21 * diff2
                    local2 = R02 * diff0 + R12 * diff1 + R22 * diff2
                    # Compute the normalized squared distance.
                    norm2 = (local0**2)/(sigma0**2) + (local1**2)/(sigma1**2) + (local2**2)/(sigma2**2)
                    density[ix, iy, iz] += np.exp(-0.5 * norm2)

# --- Step 4: Interactive Reconstruction Function ---
def update_reconstruction(scale_cutoff):
    # Compute average scale for each gaussian.
    avg_scales = np.mean(scales, axis=1)
    # Filter: keep only Gaussians with average scale <= scale_cutoff.
    indices = np.where(avg_scales <= scale_cutoff)[0]
    print(f"Using {len(indices)} out of {N} gaussians (avg scale cutoff = {scale_cutoff:.4f})")
    
    # Filter the Gaussian parameters.
    means_f = means[indices]
    scales_f = scales[indices]
    rot_matrices_f = rot_matrices[indices]
    
    # Reinitialize the density grid.
    density_filtered = np.zeros((nx, ny, nz), dtype=np.float32)
    
    # Build density field from the filtered Gaussians.
    add_gaussians_to_density(density_filtered,
                             means_f.astype(np.float32),
                             scales_f.astype(np.float32),
                             rot_matrices_f,
                             x, y, z)
    
    # Run marching cubes to extract the isosurface.
    verts, faces, normals, values = measure.marching_cubes(density_filtered, level=0.5, spacing=spacing)
    verts_world = verts + min_bound
    
    # Build the mesh.
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(verts_world)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    mesh.compute_vertex_normals()
    
    # Display the mesh.
    o3d.visualization.draw_geometries([mesh])

# --- Determine Slider Range ---
avg_scales = np.mean(scales, axis=1)
min_avg_scale = float(np.min(avg_scales))
max_avg_scale = float(np.max(avg_scales))
print(f"Average scale range: [{min_avg_scale:.4f}, {max_avg_scale:.4f}]")

# Create and display the slider widget.
slider = widgets.FloatSlider(
    value=max_avg_scale,
    min=min_avg_scale,
    max=max_avg_scale,
    step=(max_avg_scale - min_avg_scale) / 100,
    description='Scale cutoff:',
    continuous_update=False
)
widgets.interact(update_reconstruction, scale_cutoff=slider)


Average scale range: [0.0002, 0.1333]


interactive(children=(FloatSlider(value=0.13332051038742065, continuous_update=False, description='Scale cutof…

<function __main__.update_reconstruction(scale_cutoff)>